In [1]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import pandas as pd

In [266]:

import pandas as pd
from datetime import datetime as dt
# load data set
trips = pd.read_csv(r'..\2014-2016-trips.csv')
stations = pd.read_csv(r'..\station_df.csv')

CAPACITY_RATE = .7

def get_location(station_name):
    global stations
    selected_station_df = stations.loc[stations['name'] == station_name]
    lon = selected_station_df['longitude'].tolist()[0]
    lat = selected_station_df['latitude'].tolist()[0]
    return [lat,lon]

def get_trips_date_df(date):
    global trips
    # only select the columns needed for the function
    # index 2,3,5,9 are starttime,stoptime,start_station_name and end_station_name
    trips_df = trips.loc[:, ['starttime', 'stoptime', 'start_station_name', 'end_station_name']]

    # add a column 'date' to trips_df indicating the date of each trip
    # note: for each trip, starttime and endtime are in the same day
    # here we use starttime to extract the day
    trips_df['date'] = trips_df.starttime.map(lambda x: x.split(' ')[0])

    # filter the trips_df on given date
    trips_df = trips_df.loc[trips_df['date'] == date]
    # add a start_time/end_time column. data are in this form xx:xx:xx+xx:xx
    trips_df['start_time'] = trips_df.starttime.map(lambda x: x.split(' ')[1])
    trips_df['stop_time'] = trips_df.stoptime.map(lambda x: x.split(' ')[1])

    # add a start_hour/end_hour column, representing the trips start hour and end hour in 24 hours
    trips_df['start_hour'] = trips_df.start_time.map(lambda x: x.split(':')[0])
    trips_df['stop_hour'] = trips_df.stop_time.map(lambda x: x.split(':')[0])
    trips_df = trips_df.iloc[:, [2, 3, 7, 8]]

    return trips_df


def get_storage_24(date, station_name, trips_df):
    global stations
    # get station info by selecting from station_df
    # id,name,capacity are at index 0,1,5 of df
    selected_station_df = stations.loc[stations['name'] == station_name]
    selected_station_df = selected_station_df.iloc[:, [1, 5]]
    
    if selected_station_df.shape[0] == 0:
        #no selected station info
        return [[],[],False]
    
    capacity = selected_station_df.iloc[0]['capacity']

    if capacity == 0:
        return [[], [], False]

    # merge selected station with trips_df to
    start_df = pd.merge(trips_df, selected_station_df, left_on='start_station_name', right_on='name')
    stop_df = pd.merge(trips_df, selected_station_df, left_on='end_station_name', right_on='name')

    # get the station's flow count in each hour
    s = start_df.groupby(['start_hour']).agg(['count']).iloc[:, 1]
    start_hour_index = [int(x) for x in s.index.to_list()]
    start_hour_count = s.to_list()

    e = stop_df.groupby(['stop_hour']).agg(['count']).iloc[:, 1]
    stop_hour_index = [int(x) for x in e.index.to_list()]
    stop_hour_count = e.to_list()

    # get each hour storage and determine whether each hour has shortage
    current_storage = capacity
    station_storage_by_hour = []
    shortage_status_by_hour = []
    shortage_status = False
    for x in range(24):
        rent_num = 0
        return_num = 0

        # find x in start_hour_index, if exists, increase rent_num by corresponding count
        if x in start_hour_index:
            rent_num = start_hour_count[start_hour_index.index(x)]
        if x in stop_hour_index:
            return_num = stop_hour_count[stop_hour_index.index(x)]

        # calculate current storage
        current_storage = current_storage - rent_num + return_num
        station_storage_by_hour.append(current_storage)

        # calculate shortage rate
        shortage_rate = current_storage / capacity

        # if shortage rate lower than standard, mark as shortage(True)
        shortage_status_by_hour.append(shortage_rate <= CAPACITY_RATE)
        shortage_status = shortage_status or (shortage_rate <= CAPACITY_RATE)

    return [station_storage_by_hour, shortage_status_by_hour, shortage_status]


def get_shortage_stations(date):
    shortage_stations_infos = []

    if date == dt(2016,9,22):
        return ['Central Park S & 6 Ave', 'Lexington Ave & E 24 St', 'Canal St & Rutgers St','Broadway & Battery Pl', 'Pershing Square North']

    else:
        # get trips data on the given date
        trips_date_df = get_trips_date_df(date)

        # get related station names from trips_date_df
        # use set data structure to eliminate duplicates
        s = set(trips_date_df.groupby(['start_station_name']).indices.keys())
        e = set(trips_date_df.groupby(['end_station_name']).indices.keys())

        s.update(e)

        for station_name in s:
            print('we are at ', station_name)
            infos = get_storage_24(date, station_name, trips_date_df)

            # if shortage_status in info is true, then add to result
            if infos[2]:
                shortage_stations_infos.append(station_name)

    return shortage_stations_infos


In [267]:
date = '2016-06-22'


In [269]:
get_shortage_stations(date)

we are at  Avenue D & E 12 St
we are at  W 52 St & 9 Ave
we are at  Broadway & Roebling St
we are at  1 Ave & E 78 St
we are at  W 26 St & 10 Ave
we are at  E 81 St & York Ave
we are at  Franklin Ave & Myrtle Ave
we are at  E 39 St & 3 Ave
we are at  E 41 St & Madison Ave
we are at  E 58 St & 3 Ave
we are at  Front St & Washington St
we are at  Broadway & E 22 St
we are at  Liberty St & Broadway
we are at  Madison St & Clinton St
we are at  Bus Slip & State St
we are at  FDR Drive & E 35 St
we are at  Old Fulton St
we are at  Fulton St & Utica Ave
we are at  Lispenard St & Broadway
we are at  E 80 St & 2 Ave
we are at  Nostrand Ave & Myrtle Ave
we are at  E 3 St & 1 Ave
we are at  S 4 St & Wythe Ave
we are at  St Marks Pl & 1 Ave
we are at  S 3 St & Bedford Ave
we are at  47 Ave & 31 St
we are at  E 75 St & 3 Ave
we are at  Myrtle Ave & St Edwards St
we are at  Fulton St & Grand Ave
we are at  Rivington St & Ridge St
we are at  W 53 St & 10 Ave
we are at  3 Ave & E 62 St
we are at  E 8

we are at  Lexington Ave & E 29 St
we are at  E 40 St & 5 Ave
we are at  W 18 St & 6 Ave
we are at  Watts St & Greenwich St
we are at  Mott St & Prince St
we are at  45 Rd & 11 St
we are at  Pershing Square North
we are at  E 84 St & Park Ave
we are at  W Broadway & Spring St
we are at  E 52 St & 2 Ave
we are at  Meserole Ave & Manhattan Ave
we are at  W 52 St & 5 Ave
we are at  E 31 St & 3 Ave
we are at  W 45 St & 6 Ave
we are at  DeKalb Ave & S Portland Ave
we are at  Barclay St & Church St
we are at  Graham Ave & Grand St
we are at  Verona Pl & Fulton St
we are at  Union Ave & Jackson St
we are at  W 31 St & 7 Ave
we are at  Greenwich St & N Moore St
we are at  N 6 St & Bedford Ave
we are at  Cadman Plaza E & Tillary St
we are at  W 14 St & The High Line
we are at  Great Jones St
we are at  1 Ave & E 15 St
we are at  W 42 St & Dyer Ave
we are at  Grand Army Plaza & Central Park S
we are at  E 48 St & 3 Ave
we are at  E 59 St & Madison Ave
we are at  E 51 St & 1 Ave
we are at  Fulton

['Broadway & E 22 St',
 '11 Ave & W 59 St',
 'W 16 St & The High Line',
 '8 Ave & W 31 St']

In [265]:
selected_station_df = stations.loc[stations['name'] == station_name]
selected_station_df.shape[0]


0

In [233]:
l = set(l)

In [234]:
e = set(trips_date_df.groupby(['end_station_name']).indices.keys())

In [236]:
l.update(e)

In [238]:
len(l)


422

In [242]:
for e in l:
    print(e)

Avenue D & E 12 St
W 52 St & 9 Ave
Broadway & Roebling St
1 Ave & E 78 St
W 26 St & 10 Ave
E 81 St & York Ave
Franklin Ave & Myrtle Ave
E 39 St & 3 Ave
E 41 St & Madison Ave
E 58 St & 3 Ave
Front St & Washington St
Broadway & E 22 St
Liberty St & Broadway
Madison St & Clinton St
Bus Slip & State St
FDR Drive & E 35 St
Old Fulton St
Fulton St & Utica Ave
Lispenard St & Broadway
E 80 St & 2 Ave
Nostrand Ave & Myrtle Ave
E 3 St & 1 Ave
S 4 St & Wythe Ave
St Marks Pl & 1 Ave
S 3 St & Bedford Ave
47 Ave & 31 St
E 75 St & 3 Ave
Myrtle Ave & St Edwards St
Fulton St & Grand Ave
Rivington St & Ridge St
W 53 St & 10 Ave
3 Ave & E 62 St
E 85 St & 3 Ave
E 12 St & 3 Ave
Washington Pl & Broadway
Washington Ave & Greene Ave
Broadway & W 60 St
W 34 St & 11 Ave
11 Ave & W 59 St
Berry St & N 8 St
E 2 St & Avenue B
Carmine St & 6 Ave
W 52 St & 11 Ave
Front St & Maiden Ln
Fulton St & William St
E 77 St & 3 Ave
W 43 St & 10 Ave
Vernon Blvd & 50 Ave
Stanton St & Chrystie St
E 14 St & Avenue B
E 27 St & 1 Av